# 4.1.2 CSG in 2D

An alternative approach to SplineGeometry is the new CSG2d interface, which allows using constructive solid geometry definitions (similar to 3D).

This feature is new and currently has some limitations compared to `SplineGeometry`, like periodic boundaries.


## First Example

In [ ]:
from netgen.geom2d import CSG2d, Circle, Rectangle

geo = CSG2d()

# define some primitives
circle = Circle( center=(0,0), radius=1.0, mat="mat1", bc="bc_circle" )
rect = Rectangle( pmin=(0,0), pmax=(1.5,1.5), mat="mat2", bc="bc_rect" )

# use operators +, - and * for union, difference and intersection operations
domain1 = circle - rect
domain2 = circle * rect
domain2.Mat("mat3").Maxh(0.1) # change domain name and maxh
domain3 = rect-circle

# add top level objects to geometry
geo.Add(domain1)
geo.Add(domain2)
geo.Add(domain3)

# generate mesh
m = geo.GenerateMesh(maxh=0.3)

# use NGSolve just for visualization
from ngsolve.webgui import Draw
from ngsolve import Mesh, VOL
mesh = Mesh(m)
mesh.Curve(3)
cf = mesh.RegionCF(VOL, dict(mat1=0, mat2=4, mat3=7)) 
Draw(cf, mesh)


## Solid2d class

Internally, there is just one solid type (`Solid2d`), containing a closed chain of quadratic spline segments. For convenience, there are a few helper Functions, like `Rectangle` and `Circle` used above.

The general constructor of `Solid2d` expects a list of points. To define curved segments and set individual edge propierties (like boundary condition name and maxh), you can put `EdgeInfo` objects between two points, or a `PointInfo` object after a Point.

In [ ]:
from netgen.geom2d import EdgeInfo as EI, PointInfo as PI, Solid2d

geo = CSG2d()

rect = Solid2d( [
    (0,0),
    (1,0),
    PI(maxh=0.02),   # set maxh at point (1,0)
    EI(bc="right"),  # set bc for segment (1,0)-(1,1)
    (1,1),
    (0,1),
    EI(maxh=0.05)    # set maxh for segment (0,1)-(0,0)
  ], mat="rect" )

circle = Solid2d( [
    (0, -1),
    EI(( 1,  -1)), # control point for quadratic spline
    (1,0),
    EI(( 1,  1), maxh=0.05), # spline with maxh
    (0,1),
    EI((-1,  1)),
    (-1,0),
    EI((-1, -1), bc="left"), # spline with bc
    ])

# Solid2d has some methods to change it in-place:
# Move(), Scale() and Rotate()
circle.Scale(0.5).Move((2,0))
circles = circle + circle.Copy().Move((-3,0))

rect.Rotate(45, center=(0.5,0.5))

geo.Add(rect)
geo.Add(circles)

mesh = Mesh(geo.GenerateMesh())
mesh.Curve(3)
print(mesh.GetBoundaries())
Draw(mesh)

In [ ]:
help(Solid2d)

In [ ]:
help(PI)

In [ ]:
help(EI)